In [0]:
from google.colab import drive
drive.mount('/content/gdirve/')

In [0]:
a = []
while(1):
    a.append('1')

In [0]:
# import tensorflow as tf
# tf.test.gpu_device_name()
!cat /proc/meminfo

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip install Mecab
!pip install konlpy

In [0]:
!pip install EchoTorch

In [0]:
#-*- coding: utf-8 -*-
import torch
from torch import autograd, nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
%matplotlib inline
import datetime as dt
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import codecs
import re
import logging
import datetime
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from gensim.models.doc2vec import Doc2Vec as w
from gensim import utils, matutils
from numpy import exp, dot, zeros, outer, random, dtype, get_include, float32 as REAL,\
     uint32, seterr, array, uint8, vstack, argsort, fromstring, sqrt, newaxis, ndarray, empty, sum as np_sum
from konlpy.tag import Mecab
import argparse
import base64
import sys
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

In [0]:
raw_kor1 = [] #wiki
with open ('/content/gdirve/My Drive/wiki.txt', mode='r', encoding='utf-8') as file:
  lines = file.readlines()
  for line in lines:
      raw_kor1.append(line)

In [0]:
raw_kor2 = [] #sejong
with open ('/content/gdirve/My Drive/written_spoken_corpus.txt', mode='r', encoding='utf-8') as file:
  lines = file.readlines()
  for line in lines:
    parsed = line.split('.')
    for p in parsed:
      raw_kor2.append(p+'.')

In [0]:
ref_pat = r'&lt;ref\s*\w*=*&\w*;*'
web_pat = r'https*://[a-z0-9]+.[a-z0-9]+.[a-z0-9]*.[a-z0-9]*/*[a-z0-9]*'
foot_pat = r'_{1, 10}[0-9]{1, 20}'
num_pat = r'[0-9][0-9][0-9][0-9][0-9]+'

clean_li = []
for raw in raw_kor1:

    clean = re.sub('\[', '', raw)
    clean = re.sub('\]', '', clean)
    clean = re.sub('분류:', '', clean)
    clean = re.sub('</text>', '', clean)
    clean = re.sub('\n', '', clean)
    clean = re.sub(' 분류 : ', '', clean)
    clean = re.sub('썸네일', '', clean)
    clean = re.sub('웹 인용', '', clean)
    clean = re.sub(ref_pat, '', clean)
    clean = re.sub(web_pat, '', clean)
    clean = re.sub(foot_pat, '', clean)
    clean = re.sub(num_pat, '', clean)
    #clean = re.sub('\W+', ' ', clean)
    clean = re.sub('[^a-zA-Z0-9가-힣.,?!]+', ' ', clean)
    clean = re.sub('[a-zA-Z]+', ' ', clean)
    clean = re.sub('\s{2,100}', '', clean)
    clean = re.sub('.{2, 100}', '', clean)
    if clean.startswith('파일'):
      clean = None
    elif clean[-2:-1]=='|':
      clean = None
    else:
      clean_li.append(clean)

In [0]:
clean_li[0]

In [0]:
len(clean_li)

In [0]:
len(raw_kor1)

In [0]:
len(clean_li)+len(raw_kor1)

In [0]:
raw_kor3 = []
def tokenize(list1, list2):
    tagger = Mecab()
    for lines1 in list1:
    #s= " ".join(tagger.morphs(sentence))
      raw_kor3.append((' ').join(tagger.morphs(lines1)))
    for lines2 in list2:
      raw_kor3.append((' ').join(tagger.morphs(lines2)))

In [0]:
tokenize(clean_li, raw_kor2)

In [0]:
#import utils
print("Total words in text: {}".format(len(raw_kor3)))
print("Unique words: {}".format(len(set(raw_kor3)))) # `set` removes any duplicate words

In [0]:
li = [] 
for w in raw_kor3:
    word = re.sub('\n', '', w)
    li.append(word)
st = ''
for w in li:
    # word = w.split()
    st+=w
print(st[0:10])

In [0]:
import re
from collections import Counter

def preprocess(text):
    words = text.split()
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return list(zip(x_batches, y_batches))


def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [0]:
words = preprocess(st)
print(words[:30])

In [0]:
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

In [0]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5 #2.71828
word_counts = Counter(int_words)
#print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

print(train_words[:30])

In [0]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [0]:
# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # you should get some indices around the idx

In [0]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [0]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

In [0]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='gpu'):
    embed_vectors = embedding.weight
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

In [0]:
import torch
from torch import nn
import torch.optim as optim

In [0]:
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        self.in_embed = nn.Embedding(n_vocab,n_embed)
        self.out_embed = nn.Embedding(n_vocab,n_embed)
        
        self.in_embed.weight.data.uniform_(-1,1)
        self.out_embed.weight.data.uniform_(-1,1)
        
    def forward_input(self, input_words):
        input_vector = self.in_embed(input_words)
        return input_vector
    
    def forward_output(self, output_words):
        output_vector = self.out_embed(output_words)

        return output_vector
    
    def forward_noise(self, batch_size, n_samples):
        """ Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        if self.noise_dist is None:
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
        noise_words = torch.multinomial(noise_dist,
                                        batch_size * n_samples,
                                        replacement=True)
        
        device = "cuda" if model.out_embed.weight.is_cuda else "cpu"
        noise_words = noise_words.to(device)

        noise_vector = self.out_embed(noise_words).view(batch_size,n_samples,self.n_embed)        
        return noise_vector

In [0]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_vectors, output_vectors, noise_vectors):
        
        batch_size, embed_size = input_vectors.shape
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        output_vectors = output_vectors.view(batch_size, 1, embed_size)

        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)  
        return -(out_loss + noise_loss).mean()

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

word_freqs = np.array(sorted(freqs.values(), reverse=True))
unigram_dist = word_freqs/word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

embedding_dim = 300
model = SkipGramNeg(len(vocab_to_int), embedding_dim, noise_dist=noise_dist).to(device)

criterion = NegativeSamplingLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 1500
steps = 0
epochs = 5

for e in range(epochs):
    for input_words, target_words in get_batches(train_words, 512):
        steps += 1
        inputs, targets = torch.LongTensor(input_words), torch.LongTensor(target_words)
        inputs, targets = inputs.to(device), targets.to(device)

        input_vectors = model.forward_input(inputs)
        output_vectors = model.forward_output(targets)
        noise_vectors = model.forward_noise(inputs.shape[0], 5)

        loss = criterion(input_vectors, output_vectors, noise_vectors)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:
            print("Epoch: {}/{}".format(e+1, epochs))
            print("Loss: ", loss.item()) 
            valid_examples, valid_similarities = cosine_similarity(model.in_embed, device=device)
            _, closest_idxs = valid_similarities.topk(6)

            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...\n")

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [0]:
[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Malgun' in f.name]
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
f = [f.name for f in fm.fontManager.ttflist]
print(len(font_list))
# 10개의 폰트 명 만 출력
f[:10]

In [0]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.in_embed.weight.to('cpu').data.numpy()

In [0]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [0]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="/content/gdirve/My Drive/malgun.ttf").get_name()
rc('font', family=font_name)

In [0]:
path = "/content/gdirve/My Drive/malgun.ttf"
fontprop = fm.FontProperties(fname=path, size=10)
plt.style.use('seaborn-whitegrid')
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7, fontproperties=fontprop)